https://www.kaggle.com/vetrirah/janatahack-independence-day-2020-ml-hackathon/code

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the training data
import pandas as pd
my_data = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/train.csv")

In [ ]:
my_data.head()

In [ ]:
# Combine all textual data into a single column
my_data['total_text'] = my_data['TITLE'] + my_data['ABSTRACT']

In [ ]:
my_data.set_index('ID', inplace = True)
my_data.head()

In [ ]:
my_data = my_data[['total_text', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [ ]:
my_data.head()

In [ ]:
# Clean the textual column
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

In [ ]:
import re
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_tokenize(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: word_lemmatizer(x))
    df[new_text_field_name] = df[new_text_field_name].apply(lambda x: ' '.join(x))
    return df

In [ ]:
train_clean_df = clean_text(my_data, 'total_text', 'total_text_clean')

In [ ]:
final_df = train_clean_df[['total_text_clean', 'Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]
final_df.head(5)

In [ ]:
final_df['labels'] = final_df.apply(lambda x: list([x['Computer Science'], x['Physics'], x['Mathematics'], x['Statistics'], x['Quantitative Biology'], x['Quantitative Finance']]),axis=1) 
final_df.head()

In [ ]:
train_df = final_df[['total_text_clean','labels']]
train_df.columns = ['text','labels']
train_df.head()

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel

In [ ]:
# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=6,
    args={"reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 5},
)

In [ ]:
model.train_model(train_df)

In [ ]:
test_df = pd.read_csv("/kaggle/input/janatahack-independence-day-2020-ml-hackathon/test.csv")
test_df['total_text'] = test_df['TITLE'] + test_df['ABSTRACT']

In [ ]:
test_df.set_index('ID', inplace = True)
test_df.head()

In [ ]:
test_clean_df = clean_text(test_df, 'total_text', 'total_text_clean')

In [ ]:
test_clean_df = test_clean_df[['total_text_clean']]
test_clean_df.head()

In [ ]:
test_clean_df.shape

In [ ]:
prediction_list = test_clean_df.total_text_clean.tolist()

In [ ]:
predictions, raw_outputs = model.predict(prediction_list)

In [ ]:
test_clean_df['predictions'] = predictions

In [ ]:
test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']] = pd.DataFrame(test_clean_df['predictions'].tolist(), index= test_clean_df.index)

In [ ]:
test_clean_df.head()

In [ ]:
submission_df = test_clean_df[['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']]

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv("submission_df.csv")